In [2]:
import numpy as np
import pickle
import sys

In [35]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [4]:
def create_directory(path):
    if not os.path.exists(path):
        print('Creating directory: ', path)
        os.makedirs(path)

In [5]:
def process_annots(annots, min_distance):    
    count = len(annots)
    flags = np.ones((count, ))
    
    for i in range(1, count):
        if annots[i, 0] - annots[i-1, 0]<=min_distance:
            flags[i-1] = 0
            
    annots = annots[flags==1]
    #print('Annnot prcess: before, after :: ', count, len(annots))
    return annots
            
    
def process_annots_dataset(ds, min_distance):
    for subject in range(len(ds)):
        for sess in range(len(ds[subject])):            
            annots = ds[subject][sess][1]
            ds[subject][sess][1] = process_annots(annots, min_distance)            
    return ds
    
def process_uva_lab_data(data, min_distance):
    #usc:0-13, uva: 2, 5, 2, 2, 4, 5, 1
    data[14] = [data[14][0], data[15][0]]
    data[15] = [data[16][0], data[17][0], data[18][0], data[19][0], data[20][0]]
    data[16] = [data[21][0], data[22][0]]
    data[17] = [data[23][0], data[24][0]]
    data[18] = [data[25][0], data[26][0], data[27][0], data[28][0]]
    data[19] = [data[29][0], data[30][0], data[31][0], data[32][0], data[33][0]]
    data[20] = data[34]
    data = data[:21]
    data = process_annots_dataset(data, min_distance)
    return data

In [7]:
root_path ='C:/ASM/DevData/eating/eating_detection/'
with open(root_path + "data/steven_lab_data_800.pkl", 'rb') as file:
    lab_data = pickle.load(file)
with open(root_path + "data/uva_lab_data_800.pkl", 'rb') as file:
    uva_lab_data = pickle.load(file)
    uva_lab_data = process_uva_lab_data(uva_lab_data, 16)
lab_data.extend(uva_lab_data)
print('Subject counts total Lab: ', len(lab_data))

Subject counts total Lab:  28


In [8]:
with open(root_path + "data/steven_free_data_800.pkl", 'rb') as file:
    free_data = pickle.load(file)
print('Subject counts total Free: ', len(free_data))

Subject counts total Free:  11


In [10]:
def find_min_points_by_xth(x, x_th, min_bite_interval):        
    step_length = min_bite_interval//2        
    count = len(x)
    
    mp = []
    for i in range(0, count-step_length, step_length):
        min_index = i + np.argmin(x[i:i+step_length])        
        if x[min_index] <= x_th:
            mp.append(min_index)
    
    if len(mp)<=1:
        return mp
    
    while True:
        res = []
        count = len(mp)
        ix = mp[0]
        ixRight = mp[1]
        if ixRight - ix > min_bite_interval or x[ix] < x[ixRight]:
            res.append(ix)
        
        for i in range(1, count - 1):
            ix = mp[i]
            ixLeft = mp[i - 1]
            ixRight = mp[i + 1]

            cond_left = ix - ixLeft > min_bite_interval or x[ix] <= x[ixLeft]
            cond_right = ixRight - ix > min_bite_interval or x[ix] < x[ixRight]        

            if cond_left and cond_right:
                res.append(ix)
        
        ix = mp[count - 1]
        ixLeft = mp[count - 2]
        if ix - ixLeft > min_bite_interval or x[ix] <= x[ixLeft]:
            res.append(ix)            
        
        if len(mp) == len(res):
            break        
        mp = res        
    
    return mp

In [11]:
def remove_min_points_at_boundary(mp, data_len, window_size):
    half_window = window_size//2
    si, ei = 0, len(mp)-1
    mp_count = len(mp)
    
    while si<mp_count and mp[si]-half_window<0:
        si += 1
    
    while ei>=0 and mp[ei]+half_window>data_len:
        ei -= 1
        
    mp = mp[si:ei+1]
    return mp
    

In [12]:
def filter_windows_by_feature(windows, labels, var_th):    
    count = len(labels)
    flags = np.full((count,), False, dtype=bool)
    
    for i in range(count):         
        v = np.sum(np.var(windows[i, :, :], axis = 0))        
        flags[i] = (v >= var_th)
        
    windows = windows[flags, :, :]
    labels = labels[flags]
    return windows, labels 

In [13]:
def get_labels_lab(mp, annots, window_size):       
    half_window = window_size//2
    mp_count = len(mp)    
    if mp_count == 0:
        return mp
    
    labels = np.zeros((mp_count, ))      
    annot_count = len(annots)
    if annot_count==0:
        return labels
    
    annot_index = 0
    for i in range(mp_count):
        left_index, right_index = mp[i] - half_window, mp[i] + half_window-1        
        
        while left_index > annots[annot_index, 0]:
            annot_index += 1
            if annot_index == annot_count:
                return labels            
            
        if left_index <= annots[annot_index, 0] <= right_index:
            labels[i] = annots[annot_index, 1]           
            
    return labels

In [14]:
def get_labels_free(mp, annots, window_size):       
    half_window = window_size//2
    mp_count = len(mp)
    if mp_count == 0:
        return mp
    
    labels = np.zeros((mp_count, ))      
    annot_count = len(annots)
    if annot_count==0:
        return lables
    
    annot_index = 0
    for i in range(mp_count):                        
        while mp[i] > annots[annot_index, 1]:
            annot_index += 1
            if annot_index == annot_count:
                return labels
            
        if mp[i] >= annots[annot_index, 0]:
                labels[i] = annots[annot_index, 2]            
        
    return labels

In [15]:
def get_windows(data, mp, window_size):    
    half_window = window_size//2
    mp_count = len(mp)
    windows = np.zeros((mp_count, window_size, data.shape[1]))    
    
    for i in range(mp_count):    
        ix = mp[i]        
        windows[i, :, :] = data[ix-half_window:ix+half_window, :]
        
    return windows            

In [22]:
def test_for_subject_sess_lab(ds, models, subject, sess, window_size, x_th, var_th, min_bite_interval):    
    
    half_window = window_size//2       
    model = models[subject]
    data = ds[subject][sess][0]
    annots = ds[subject][sess][1]            
    data = data[:, 1:]            
    print("Actual Sample count, Bites, Sips, total: ", data.shape, np.sum(annots[:,1]==1), np.sum(annots[:,1]==2), annots.shape)


    mp = find_min_points_by_xth(data[:, 0], x_th, min_bite_interval)            
    mp = remove_min_points_at_boundary(mp, len(data), window_size)    
    windows = get_windows(data, mp, window_size)
    
    mp = np.array(mp)
    windows, mp = filter_windows_by_feature(windows, mp, var_th)
    labels = get_labels_lab(mp, annots, window_size)
    
    windows = (windows+9.8)/(2*9.8)
    windows[windows>1]=1
    windows[windows<0]=0
    windows = windows.reshape((windows.shape[0], windows.shape[1], windows.shape[2], 1))
    
    res = model.predict(windows, verbose=0)    
    res_pos = res[:,0]<0.5
    labels_pos = labels>0
    res_neg = np.logical_not(res_pos)
    labels_neg = np.logical_not(labels_pos)    
    
    tn = np.sum(np.logical_and(res_neg, labels_neg))
    tp = np.sum(np.logical_and(res_pos, labels_pos))    
    fp = np.sum(np.logical_and(res_pos, labels_neg))
    fn = np.sum(np.logical_and(res_neg, labels_pos))    
    p = tp/(tp+fp)
    r = tp/(tp+fn)
    f = 2*p*r/(p+r)    
    res1 = [tn, tp, fp, fn, p, r, f]
    
    conf_mat = np.zeros((3,3))
    res = res[res_pos]        
    mp = mp[res_pos]
    labels = np.ones((len(mp),), dtype=int)
    labels[res[:,1]<res[:,2]] = 2
    
    ix = 0
    for i in range(len(annots)):
        label = annots[i,1]
        while ix < len(mp):
            if mp[ix]-half_window <= annots[i,0] <= mp[ix]+half_window:                
                label2 = labels[ix]                
                conf_mat[label, label2] += 1                
                labels[ix] = 0     
                break
                
            elif mp[ix]+half_window < annots[i,0]:
                ix+=1
            
            elif mp[ix]-half_window > annots[i,0]:
                conf_mat[label, 0] += 1
                break
            
        if ix == len(mp):
            conf_mat[label, 0] += 1
    
    conf_mat[0, 1] = np.sum(labels==1)
    conf_mat[0, 2] = np.sum(labels==2)
    
    return len(annots), len(mp), res1,conf_mat

In [42]:
from keras.models import load_model
weighted = True
include_free_data = False
path = 'K:/ASM/projects/eating/results/window_'+str(window_size)+'_free_'+str(int(include_free_data))+'_weighted_'+str(int(weighted))+'/'

weights = [1, 1, 1]
models = {}
for subject in range(-1, len(lab_data)):        
    print('Loading model Subject >>', subject)
    model_path = path+'models/subject_'+str(subject)+'.h5'
    if weighted:
        model = load_model(model_path, custom_objects= {'weighted_categorical_crossentropy': weighted_categorical_crossentropy, 'weights':weights, 'loss':loss})
    else:
        model = load_model(model_path)
        
    models[subject] = model


Loading model Subject >> -1


NameError: name 'loss' is not defined

In [36]:
x_th = -3
var_th = 1
min_bite_interval=32
window_size = 6*16

res = []
conf_mat = np.zeros((3,3))
for subject in range(len(lab_data)):    
    for sess in range(len(lab_data[subject])):
        print("\nSubject, Sess: ", subject, sess)
        annot_count, mp_count, r, cm = test_for_subject_sess_lab(lab_data, models, subject, sess, window_size, x_th, var_th, min_bite_interval) 
        print(r)
        print(cm)
        
        res.append(r)
        conf_mat += cm

res = np.array(res)


Subject, Sess:  0 0
   Actual Sample count, Bites, Sips, total:  (374286, 3) 320 60 (380, 2)
[1279, 132, 13, 299, 0.9103448275862069, 0.3062645011600928, 0.45833333333333326]
[[  0.   9.   6.]
 [214. 106.   0.]
 [ 36.   0.  24.]]

Subject, Sess:  1 0
   Actual Sample count, Bites, Sips, total:  (345437, 3) 32 10 (42, 2)
[563, 26, 5, 18, 0.8387096774193549, 0.5909090909090909, 0.6933333333333334]
[[ 0.  5.  5.]
 [20. 12.  0.]
 [ 1.  0.  9.]]

Subject, Sess:  2 0
   Actual Sample count, Bites, Sips, total:  (367241, 3) 132 11 (143, 2)
[796, 58, 12, 86, 0.8285714285714286, 0.4027777777777778, 0.5420560747663552]
[[ 0. 22.  1.]
 [88. 44.  0.]
 [ 8.  0.  3.]]

Subject, Sess:  2 1
   Actual Sample count, Bites, Sips, total:  (339363, 3) 33 9 (42, 2)
[969, 19, 12, 29, 0.6129032258064516, 0.3958333333333333, 0.4810126582278481]
[[ 0. 11.  1.]
 [15. 17.  1.]
 [ 8.  0.  1.]]

Subject, Sess:  3 0
   Actual Sample count, Bites, Sips, total:  (351300, 3) 106 3 (109, 2)
[582, 25, 11, 57, 0.69444444

[25, 24, 1, 47, 0.96, 0.3380281690140845, 0.5]
[[ 0.  2.  0.]
 [33. 13.  0.]
 [14.  1.  9.]]

Subject, Sess:  26 1
   Actual Sample count, Bites, Sips, total:  (9087, 3) 54 3 (57, 2)
[38, 21, 0, 36, 1.0, 0.3684210526315789, 0.5384615384615384]
[[ 0.  0.  0.]
 [36. 18.  0.]
 [ 0.  0.  3.]]

Subject, Sess:  26 2
   Actual Sample count, Bites, Sips, total:  (8304, 3) 38 2 (40, 2)
[19, 29, 0, 16, 1.0, 0.6444444444444445, 0.7837837837837839]
[[ 0.  4.  0.]
 [14. 24.  0.]
 [ 1.  0.  1.]]

Subject, Sess:  26 3
   Actual Sample count, Bites, Sips, total:  (7228, 3) 32 2 (34, 2)
[23, 12, 0, 26, 1.0, 0.3157894736842105, 0.4799999999999999]
[[ 0.  0.  0.]
 [21. 11.  0.]
 [ 1.  0.  1.]]

Subject, Sess:  26 4
   Actual Sample count, Bites, Sips, total:  (7833, 3) 20 17 (37, 2)
[9, 26, 0, 12, 1.0, 0.6842105263157895, 0.8125000000000001]
[[ 0.  0.  0.]
 [ 7. 13.  0.]
 [ 4. 11.  2.]]

Subject, Sess:  27 0
   Actual Sample count, Bites, Sips, total:  (9591, 3) 57 0 (57, 2)
[11, 46, 0, 12, 1.0, 0.793103

In [37]:
print(conf_mat)
r = np.sum(res, axis=0)
print(r)

[[   0.  264.   72.]
 [1667. 1303.   28.]
 [ 430.   24.  441.]]
[9300.         1920.          212.         2032.           42.91481681
   25.25969556   30.70517631]


In [38]:
c = conf_mat
p = c[1,1]/np.sum(c[:,1])
r = c[1,1]/np.sum(c[1,:])
f = 2*p*r/(p+r)
print("Bite: ", p, r, f)

p = c[2,2]/np.sum(c[:,2])
r = c[2,2]/np.sum(c[2,:])
f = 2*p*r/(p+r)
print("Sip: ", p, r, f)

p = (c[1,1] + c[1,2] + c[2,1] +c[2,2])/np.sum(c[:,1:3])
r = (c[1,1] + c[1,2] + c[2,1] +c[2,2])/np.sum(c[1:3,:])
f = 2*p*r/(p+r)
print("Combined: ", p, r, f)


Bite:  0.8189817724701446 0.4346230820547031 0.5678797123556331
Sip:  0.8151571164510166 0.49273743016759775 0.6142061281337047
Combined:  0.8424015009380863 0.46134086822501924 0.5961825726141078
